In [1]:
%load_ext autoreload
%autoreload 2

# Registering our Model Context Protocol (MCP) service.

Nacos registers an MCP service automatically **if** you use their MCP wrapper in your code:

---
```
from nacos_mcp_wrapper.server.nacos_mcp import NacosMCP
from nacos_mcp_wrapper.server.nacos_settings import NacosSettings

nacos_settings = NacosSettings()
nacos_settings.SERVER_ADDR = "127.0.0.1:8848" # <nacos_server_addr> e.g. 127.0.0.1:8848
nacos_settings.USERNAME=os.environ["NACOS_USERNAME"]
nacos_settings.PASSWORD=os.environ["NACOS_PASSWORD"]
mcp = NacosMCP("forex-tool", nacos_settings=nacos_settings, port=18001)
```
---

Now define your `@mcp.tool` as usual and run it as usual.

We've defined ours in the `forex_tool.py` file in the root directory. Simply run:
```
python forex_tool.py
```

to spin up your MCP tool and register it in Nacos.

# Registering an Agent Communication Protocol (ACP) service

First ensure that your ACP tool is up and running by executing the following command in your terminal from the root directory:
```
python search_tool.py
```

In [2]:
SERVER_ADDRESSES = "http://localhost:8848"
NAMESPACE = "public"
group = "DEFAULT_GROUP"

In [3]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

#### Do not use Nacos' Python v1 SDK!

In [ ]:
# import nacos

# SERVER_ADDRESSES = "http://localhost:8848"
# NAMESPACE = "public"

# client = nacos.NacosClient(
#     SERVER_ADDRESSES,
#     namespace=NAMESPACE,
#     # if you need auth:
#     # ak="yourAccessKey", sk="yourSecretKey"
# )

In [3]:
# data_id = "config.nacos"
# group = "DEFAULT_GROUP"
# print(client.get_config(data_id, group))

# intentionally left blank


In [11]:
# import os

# SERVICE_NAME = "internet-search-agent"
# SERVICE_IP   = os.getenv("SERVICE_IP", "127.0.0.1")
# SERVICE_PORT = int(os.getenv("SERVICE_PORT", "8008"))

# success = client.add_naming_instance(
#     service_name=SERVICE_NAME,
#     ip=SERVICE_IP,
#     port=SERVICE_PORT, 
#     weight=1.0, #A float number for load balancing weight
#     ephemeral=False, 
#     enable=True,
#     healthy=True,
#     metadata={"protocol": "acp"}
# )
# print(f"Registered with Nacos? {success}")

Registered with Nacos? True


Deregister instance

In [45]:
# SERVICE_NAME = "internet-search-agent"
# SERVICE_IP   = os.getenv("SERVICE_IP", "127.0.0.1")
# SERVICE_PORT = int(os.getenv("SERVICE_PORT", "8008"))

# success = client.remove_naming_instance(
#     service_name=SERVICE_NAME,
#     ip=SERVICE_IP,
#     port=SERVICE_PORT, 
# )
# print(f"De-Registered with Nacos? {success}")

De-Registered with Nacos? True


### Use Nacos V2 SDK

In [4]:
import os
import nest_asyncio
import warnings
from v2.nacos import (
    NacosNamingService, 
    ClientConfig, 
    ListServiceParam, 
    RegisterInstanceParam,
    DeregisterInstanceParam,
    ListInstanceParam,
    GetServiceParam,
    SubscribeServiceParam,
)
warnings.filterwarnings("ignore")
nest_asyncio.apply()

config = ClientConfig(
    server_addresses=SERVER_ADDRESSES, 
    namespace_id=NAMESPACE,
    username=os.environ["NACOS_USERNAME"],
    password=os.environ["NACOS_PASSWORD"]
)

/Users/tituslim/Documents/Personal Learning Folder/Personal Projects/nacos-playground/.venv/lib/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.31.1 at nacos_grpc_service.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(


In [ ]:
named_service = await NacosNamingService.create_naming_service(config)

To register a service!

In [6]:
SERVICE_NAME = "internet-search-agent"
SERVICE_IP   = os.getenv("SERVICE_IP", "127.0.0.1")
SERVICE_PORT = int(os.getenv("SERVICE_PORT", "8008"))

In [7]:
success = await named_service.register_instance(
    RegisterInstanceParam(
        ip = SERVICE_IP,
        port = SERVICE_PORT,
        weight = 1.0, #for load balancing
        enabled = True,
        healthy = True,
        service_name = SERVICE_NAME,
        metadata = {
            'deployment_type': 'ACP',
            'description': 'This agent searches the internet',
            'llm_used': 'gpt-4o-mini',
            'llm_vendor': 'OpenAI',
            'framework': 'LangGraph',
            'author': 'Titus Lim',
            'email': 'abc123@email.com',
        }
    )
)

print(success)

True


To batch register a service:

---
```
from v2.nacos import BatchRegisterInstanceParam

success = await named_service.batch_register_instances(
    BatchRegisterInstanceParam(
        service_name = SERVICE_NAME,
        instances = [
            RegisterInstanceParam(
                ...
            ),
            RegisterInstanceParms(
                ...
            )
        ]
    )
)
```
---

Update service

In [15]:
agent_name = 'tavily_search_agent'

In [8]:
success = await named_service.update_instance(
     RegisterInstanceParam(
        ip = SERVICE_IP,
        port = SERVICE_PORT,
        weight = 1.0, #for load balancing
        enabled = True,
        healthy = True,
        service_name = SERVICE_NAME,
        metadata = {
            'deployment_type': 'ACP',
            'description': 'This agent searches the internet',
            'agent_name': agent_name, #add agent_name
            'llm_used': 'gpt-4o-mini',
            'llm_vendor': 'OpenAI',
            'framework': 'LangGraph',
            'author': 'Titus Lim',
            'email': 'abc123@email.com',
        }
    )
)

print(success)

True


See services available

In [9]:
await named_service.list_services(
    ListServiceParam(namespace_id=NAMESPACE)
)

ServiceList(count=2, services=['forex-tool::1.9.2', 'internet-search-agent'])

Find metadata on services

In [10]:
await named_service.list_instances(
    ListInstanceParam(
        service_name = SERVICE_NAME,
        healthy_only = True,
    )
)

[Instance(instanceId='127.0.0.1#8008##DEFAULT_GROUP@@internet-search-agent', ip='127.0.0.1', port=8008, weight=1.0, healthy=True, enabled=True, ephemeral=True, clusterName='DEFAULT', serviceName='DEFAULT_GROUP@@internet-search-agent', metadata={'deployment_type': 'ACP', 'llm_vendor': 'OpenAI', 'framework': 'LangGraph', 'author': 'Titus Lim', 'description': 'This agent searches the internet', 'llm_used': 'gpt-4o-mini', 'email': 'abc123@email.com'})]

Get service parameters

In [11]:
result = await named_service.get_service(
    GetServiceParam(
        service_name=SERVICE_NAME,
    )
)

In [12]:
result.model_dump()

{'name': 'internet-search-agent',
 'groupName': 'DEFAULT_GROUP',
 'clusters': '',
 'cacheMillis': 10000,
 'hosts': [{'instanceId': '127.0.0.1#8008##DEFAULT_GROUP@@internet-search-agent',
   'ip': '127.0.0.1',
   'port': 8008,
   'weight': 1.0,
   'healthy': True,
   'enabled': True,
   'ephemeral': True,
   'clusterName': 'DEFAULT',
   'serviceName': 'DEFAULT_GROUP@@internet-search-agent',
   'metadata': {'deployment_type': 'ACP',
    'llm_vendor': 'OpenAI',
    'framework': 'LangGraph',
    'author': 'Titus Lim',
    'description': 'This agent searches the internet',
    'llm_used': 'gpt-4o-mini',
    'email': 'abc123@email.com'}}],
 'lastRefTime': 1754488411592,
 'checksum': '',
 'allIps': False,
 'reachProtectionThreshold': False,
 'jsonFromServer': ''}

In [13]:
instance = result.model_dump()['hosts'][0]
ip_address = f'{instance['ip']}:{instance['port']}'

#### And thus discoverability was born! Let's ping our "discovered" ACP agent

In [17]:
from acp_sdk.client import Client
from acp_sdk.models import MessageCompletedEvent, GenericEvent

async with Client(base_url=f'http://{ip_address}') as client:
    async for event in client.run_stream(
        agent=agent_name, input="What is a qubit??"
    ):
        match event:
            case MessageCompletedEvent():
                print("\nFinal response:", event.message)
            case GenericEvent():
                print(event.generic.update)

['agent', {'messages': [{'content': 'A qubit, or quantum bit, is the fundamental unit of quantum information in quantum computing. Unlike a classical bit, which can be either 0 or 1, a qubit can exist in a state that is a superposition of both 0 and 1. This means that a qubit can represent multiple states simultaneously, allowing quantum computers to perform complex calculations more efficiently than classical computers.\n\nKey properties of qubits include:\n\n1. **Superposition**: A qubit can be in a state of 0, 1, or both at the same time, represented mathematically as a linear combination of the two states.\n\n2. **Entanglement**: Qubits can be entangled, meaning the state of one qubit is directly related to the state of another, no matter the distance between them. This property is crucial for quantum communication and computation.\n\n3. **Interference**: Quantum algorithms often rely on the interference of probability amplitudes, which can enhance the probability of correct outcom

#### Subscribe to the service
Nacos does these when you subscribe to a service:

1. Registers a Listener
>When you call NacosClient.subscribe(service_name, listener_fn, …), the SDK tells the Nacos server “please notify me” about any changes to the instance list for service_name 

2. Maintains a Local Cache
>The SDK keeps a cached list of all currently healthy instances for that service. Upon initial subscription, it fetches the full list via HTTP or gRPC, then stores it locally.

3. Receives Push Notifications
>Nacos sends heartbeat‐driven notifications (or long‐polling responses) back to your client whenever an instance is registered, deregistered, or its health status changes. Your listener_fn(event, instance) is invoked with details of what change

Source: https://ost.51cto.com/posts/13026

In [18]:
await named_service.subscribe(
    SubscribeServiceParam(service_name = SERVICE_NAME)
)

Unsubscribe to the service

In [19]:
await named_service.unsubscribe(
    SubscribeServiceParam(service_name = SERVICE_NAME)
)

Deregister an instance

In [20]:
success = await named_service.deregister_instance(
    DeregisterInstanceParam(
        ip = SERVICE_IP,
        port = SERVICE_PORT,
        service_name = SERVICE_NAME,
        ephemeral = True,
    )
)

print(success)

True


# A shadow of MCP-Zero

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

mcp_client = MultiServerMCPClient(
    {
        "mcp_router": {
            # Make sure you start your weather server on port 8000
            "url": "http://localhost:8000/sse/",
            "transport": "sse",
        }
    }
)
tools = await mcp_client.get_tools()

In [8]:
tools

[StructuredTool(name='search_mcp_server', description='执行任务前首先使用本工具。根据任务描述及关键字搜索mcp server, 制定完成任务的步骤。注意：任务描述及关键字需要同时包含中文和英文。', args_schema={'type': 'object', 'required': ['task_description', 'key_words'], 'properties': {'task_description': {'type': 'string', 'description': '用户中文和英文任务描述，中英文描述各占单独一行。如果任务描述只包含中文，请同时输入英文描述，反之亦然。'}, 'key_words': {'type': 'string', 'description': '用户任务关键字，可以为多个，最多为4个，包含中英文关键字，英文逗号分隔'}}}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x114077920>),
 StructuredTool(name='add_mcp_server', description='安装指定的mcp server', args_schema={'type': 'object', 'required': ['mcp_server_name'], 'properties': {'mcp_server_name': {'type': 'string', 'description': 'MCP Server名称'}}}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x117c58e00>),
 StructuredTool(name='use_tool', description='使用某个MCP Server的工具', args_schema={'type': 'object'

Wow the description is in Chinese! 

In [ ]:
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI

_ = load_dotenv(find_dotenv())
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)
openai_agent = create_react_agent(model=llm, tools=tools)

In [20]:
async for chunk in openai_agent.astream(
    {"messages": [{"role": "user", "content": "What is the exchange rate between USD and EUR?"}]},
    stream_mode="updates"
):
    for value in chunk.items():
        value[-1]['messages'][0].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  search_mcp_server (call_FCyQ5AcxF8bxKYUPsf0yoroH)
 Call ID: call_FCyQ5AcxF8bxKYUPsf0yoroH
  Args:
    task_description: What is the exchange rate between USD and EUR?
    key_words: exchange rate, USD, EUR
================================= Tool Message =================================
Name: search_mcp_server

## 获取What is the exchange rate between USD and EUR?的步骤如下：
### 1. 当前可用的mcp server列表为：{"forex-tool": {"name": "forex-tool", "description": "forex-tool"}}
### 2. 从当前可用的mcp server列表中选择你需要的mcp server调add_mcp_server工具安装mcp server
================================== Ai Message ==================================
Tool Calls:
  add_mcp_server (call_OqiJFFE4afkbY7GYPTNuK47B)
 Call ID: call_OqiJFFE4afkbY7GYPTNuK47B
  Args:
    mcp_server_name: forex-tool
================================= Tool Message =================================
Name: add_mcp_server

1. forex-tool安装完成, tool 列表为: [{"name": "get_

The MCP router tool handles the routing for us! We only need to provide it the MCP tool!

## Appendix: ACP to MCP

The MCP feature looks really good! So why not we convert ACP to MCP?

First, install the dependencies: `pip install acp-mcp`

Then run 
`uvx acp-mcp localhost:8008`

> IBM's `acp-mcp` framework only supports stdio transport

In [15]:
acp_mcp_client = MultiServerMCPClient(
    {
        "search_internet_tool": {
            "command": "uvx",
            "args": [
                "acp-mcp",
                "http://localhost:8008"
            ],
            "transport": "stdio",
        },
    }
)

In [17]:
acp_mcp_tools = await acp_mcp_client.get_tools()
acp_mcp_tools

[StructuredTool(name='list_agents', description='Lists available agents', args_schema={'properties': {}, 'title': 'EmptySchema', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x121d9b560>),
 StructuredTool(name='run_agent', description='Runs an agent', args_schema={'$defs': {'AnyModel': {'additionalProperties': True, 'properties': {}, 'title': 'AnyModel', 'type': 'object'}, 'CitationMetadata': {'description': 'Represents an inline citation, providing info about information source. This\nis supposed to be rendered as an inline icon, optionally marking a text\nrange it belongs to.\n\nIf CitationMetadata is included together with content in the message part,\nthe citation belongs to that content and renders at the MessagePart position.\nThis way may be used for non-text content, like images and files.\n\nAlternatively, `start_index` and `end_index` may define a text range,\ncounting characters in the

Notice that the name of these tools are extremely generic - every ACP-MCP deployment will have them. This will present a problem once multiple ACP-MCP tools become tagged to an LLM.

In [23]:
acp_mcp_agent = create_react_agent(model=llm, tools=acp_mcp_tools)

async for chunk in acp_mcp_agent.astream(
    {"messages": [{"role": "user", "content": "What is the current stock price of NVIDIA?"}]},
    stream_mode="updates"
):
    for value in chunk.items():
        value[-1]['messages'][0].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  list_agents (call_u0B3kMd39NhFjZWzCluOHTdJ)
 Call ID: call_u0B3kMd39NhFjZWzCluOHTdJ
  Args:
================================= Tool Message =================================
Name: list_agents


================================== Ai Message ==================================
Tool Calls:
  run_agent (call_ieziZvd0frc9DCE6mJIs64Nj)
 Call ID: call_ieziZvd0frc9DCE6mJIs64Nj
  Args:
    agent: stock_price_agent
    input: [{'role': 'user', 'parts': [{'content': 'What is the current stock price of NVIDIA?'}]}]
================================= Tool Message =================================
Name: run_agent

Error: ToolException("Client.session() got an unexpected keyword argument 'session_id'. Did you mean 'session'?")
 Please fix your mistakes.
================================== Ai Message ==================================
Tool Calls:
  list_agents (call_dhfcUzjkKe3pZtcRTxmG9R5j)
 Call ID: call_dhfcU

And our agent failed to run something - our tool names are just not good enough. It tried calling a stock_price_agent which doesn't exist. We only have an internet search agent!